In [1]:
import torch
from torchvision import transforms, datasets
import numpy as np
torch.set_grad_enabled(False)
from torch import nn
# device_ids = [0, 1, 2, 3]
device_ids = [4, ]
torch.cuda.set_device(device_ids[0])
torch.multiprocessing.set_sharing_strategy('file_system')
%load_ext autoreload
%autoreload 2

In [2]:
from models.basic_template import TrainTask

dataset_name = 'cifar10'
# dataset_name = 'imagenet'
test_transform = transforms.Compose([
    # transforms.Resize(256),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    TrainTask.normalize(dataset_name)
])
train_dataset = TrainTask.create_dataset(
    data_root='/home/zzhuang/DATASET/clustering/',
    dataset_name=dataset_name,
    train=True,
    transform=test_transform,
    memory=True,
)[0]
# dataset = torch.utils.data.ConcatDataset([create_dataset(), create_dataset(False)])
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=1024,
                                           shuffle=False,
                                           num_workers=64)
test_dataset = TrainTask.create_dataset(
    data_root='/home/zzhuang/DATASET/clustering/',
    dataset_name=dataset_name,
    train=False,
    transform=test_transform,
    memory=True,
)[0]
# dataset = torch.utils.data.ConcatDataset([create_dataset(), create_dataset(False)])
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          # batch_size=256,
                                          batch_size=1024,
                                          shuffle=False,
                                          num_workers=40)
num_classes = len(np.unique(train_dataset.targets))
num_classes

10

In [4]:
from models.propos.byol_wrapper import BYOLWrapper
from network import backbone_dict
# backbone = 'resnet50'
backbone = 'bigresnet18'
encoder_type, dim_in = backbone_dict[backbone]
encoder = encoder_type()
byol = BYOLWrapper(encoder,
                 num_cluster=num_classes,
                 in_dim=dim_in,
                 temperature=0.5,
                 hidden_size=4096,
                 fea_dim=256,
                 byol_momentum=0.999,
                 symmetric=True,
                 shuffling_bn=True,
                 latent_std=0.001)
checkpoint = ''
msg = byol.load_state_dict(torch.load(checkpoint, map_location='cpu')['byol'], strict=False)
print(msg)
encoder = nn.Sequential(byol.encoder_k, byol.projector_k)
encoder = nn.DataParallel(encoder, device_ids=device_ids).cuda().eval()

<All keys matched successfully>


In [5]:
from utils import extract_features
mem_features, mem_labels = extract_features(encoder, train_loader)
test_features, test_labels = extract_features(encoder, test_loader)

100%|██████████| 49/49 [00:13<00:00,  3.56it/s]


func: 'extract_features' took a total of 13.9047 sec to run


100%|██████████| 10/10 [00:05<00:00,  1.86it/s]


func: 'extract_features' took a total of 5.4207 sec to run


In [ ]:
kwargs = {
    'metric': 'cosine',
    'distributed': True,
    'random_state': 0,
    'n_clusters': int(mem_labels.max()) + 1,
    'verbose': True
}
from torch_clustering import PyTorchKMeans, evaluate_clustering
clustering_model = PyTorchKMeans(init='k-means++', max_iter=300, tol=1e-4, **kwargs)

psedo_labels = clustering_model.fit_predict(mem_features)
cluster_centers = clustering_model.cluster_centers_

In [8]:
results = evaluate_clustering(mem_labels.cpu().numpy(), psedo_labels.cpu().numpy(), eval_metric=['nmi'])

In [ ]:
results